In [ ]:
#Code to fetch Audio features from Spotify API

In [2]:
#Install the spotipy module.
!pip install spotipy

     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 153kB 14.1MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
#Importing required modules
import spotipy
import os
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import DataFrame, json_normalize
import json


In [ ]:
track_uri_toquery = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/my_data/combined_csv.csv")

In [4]:
#Reading Mpd data for getting the track_uri list which audio files needs to be generated.
def read_mpd_data(path):

    #Creating empty Dataframe with required Columns
    tracks_df = pd.DataFrame(columns=['pid','track_uri'])

    # Fetching the data from the mpd data folder
    folderpath= path
    json_files = [pos_json for pos_json in os.listdir(folderpath) if pos_json.endswith('.json')]
    json_files.sort()

    # Fetching pid and track_uri for all the playlists
    for index, js in enumerate(json_files):
        with open(os.path.join(folderpath, js)) as json_file:
            j = json.load(json_file)

            # extracting tracks from playlists in each slice
            tracks = json_normalize(j['playlists'], record_path='tracks',
                                meta=['pid'])
            # append tracks to tracks_df
            tracks = tracks[['pid', 'track_uri']]
            tracks_df = tracks_df.append(tracks)

    #Fecthing the only track_uri from the dataframe.
    onlyURI = tracks_df['track_uri']
    
    #Wriiting the Track_uri data frame to csv.
    onlyURI.to_csv('/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/2606/trackURI.csv', index= False)

In [ ]:
# Getting the mpd Data
read_mpd_data('/content/drive/MyDrive/Somedata')

In [18]:
# Initialize Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id='6c3a43eeaea14010ac2b71a258c2103b',
                                                      client_secret='937fd98bf98242b28a3113d4c46289a4')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=40)

# Reading the Track_uri List
track_uri_list = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/2604SomeData/trackURI.csv")

# # Connsidering only the track_uri data
list_to_query = list(track_uri_list['track_uri'])

#Fetching the unique track URI columns
unique_tracks = list(set(list_to_query))
print('No. of unique Tracks : ',   len(unique_tracks ))

#Fetching the spotify audio features, in chunks of 100 uri at a time.
start = 0
end = len(unique_tracks)
for i in range(start, end, 100):
    temp_list = unique_tracks[i:i+100]
    playlists = sp.audio_features(temp_list)
    # Adding audio feature to one list.
    if "itemlist" in locals():
        itemlist = itemlist + playlists 
    else:
        itemlist = playlists

No. of unique Tracks :  407591


In [22]:
# Remove the any none values if present
list_new = []
for i in itemlist:
  if i != None:
   list_new.append(i)
   

In [24]:
#Writtinng the audio features in csv file if the list length is greater than 1.
if len(list_new1) > 0: 
    df = DataFrame (list_new1,columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','acousticness','instrumentalness','liveness','valence','tempo','type','id','uri','track_href','analysis_url','duration_ms','time_signature'])
    temp_filename = "audio_features_40K.csv" 
    path ="/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/my_data/MetaData/track_feature_new" +temp_filename;
    df.to_csv(path, index= False)

In [29]:
#Fetching the track features.
trackFeatures = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/my_data/MetaData/track_feature_newaudio_features_40K.csv')

In [26]:
#Fetching the pid and track list from the User based algorithm implementation.
pidTrackInfo = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/2604/pidTrack.csv')

In [30]:
#Mergig the pid and track details into single df
merged_df = pd.merge( pidTrackInfo,trackFeatures, left_on='track_uri', right_on='uri' , how ='inner')

In [ ]:
#Writting the merged file into the csv.
merged_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/2604/Pid_track_merge.csv')